In [1]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Make sure clone at root
!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

import os
os.chdir('gif-705-projet')

fatal: destination path 'gif-705-projet' already exists and is not an empty directory.


In [3]:
import time

import torch

from dataset import *
from models import *
import models_parts

torch.manual_seed(111)

In [4]:
N_EPOCH = 1
BATCH_SIZE = 4

In [5]:
class Downsample:
  def __init__(self):
    self.pool = nn.AvgPool2d(2)
    
  def __call__(self, x, target):
    x = self.pool(x.unsqueeze(0)).squeeze(0)
    target = self.pool(target)
    return x, target

In [6]:
dataset = dcm_dataset('../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/data', transforms=Downsample())
print("Found {} images".format(len(dataset.img_files)))

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

Found 26684 images


In [7]:
import torch.nn.functional as F

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
model = UNet(1, 1).to("cuda:0")
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

## Train 1

In [ ]:
# Test inference
timer = time.time()
for epoch in range(N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 4794.29


In [ ]:
checkpoint_path = "unet.pt"
torch.save(model.state_dict(), checkpoint_path)

In [ ]:
checkpoint_path = "/content/drive/My Drive/unet_drive.pt"
torch.save(model.state_dict(), checkpoint_path)

In [ ]:
checkpoint_path = "/content/drive/My Drive/out_images_after_train.pt"
torch.save(out, checkpoint_path)

## Train 2

In [ ]:
dataloaders = {
  'train': train_loader,
  'val': test_loader
}

In [ ]:
from collections import defaultdict


# Source: A survey of loss functions for semantic segmentation https://arxiv.org/pdf/2006.14822.pdf
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()


def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = torch.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))


def train_model(model, optimizer, scheduler, num_epochs = 25, checkpoint_path = "checkpoint.pt"):
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, (labels, bounding_box) in dataloaders[phase]:
                
                inputs = inputs.to(device)
                #labels = labels.to(device)
                bounding_box = bounding_box.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, bounding_box, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            if phase == 'train':
              scheduler.step()
              for param_group in optimizer.param_groups:
                  print("LR", param_group['lr'])

            # save the model weights
            if phase == 'val' and epoch_loss < best_loss:
                print(f"saving best model to {checkpoint_path}")
                best_loss = epoch_loss
                torch.save(model.state_dict(), checkpoint_path)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(torch.load(checkpoint_path))
    return model

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
import models_parts

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device', device)

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)

checkpoint_path = "/content/drive/My Drive/intro_ml/unet_train_8_epochs.pt"

model = train_model(model, optimizer_ft, exp_lr_scheduler, 8, checkpoint_path)

device cuda
Epoch 0/7
----------
train: bce: 0.089442, dice: 0.903759, loss: 0.496600
LR 0.0001
val: bce: 0.078562, dice: 0.902831, loss: 0.490696
saving best model to /content/drive/My Drive/intro_ml/unet_train_8_epochs.pt
77m 56s
Epoch 1/7
----------
train: bce: 0.083252, dice: 0.895242, loss: 0.489247
LR 0.0001
val: bce: 0.077742, dice: 0.897640, loss: 0.487691
saving best model to /content/drive/My Drive/intro_ml/unet_train_8_epochs.pt
78m 4s
Epoch 2/7
----------
train: bce: 0.079627, dice: 0.890216, loss: 0.484921
LR 0.0001
val: bce: 0.075268, dice: 0.894736, loss: 0.485002
saving best model to /content/drive/My Drive/intro_ml/unet_train_8_epochs.pt
78m 9s
Epoch 3/7
----------
train: bce: 0.182582, dice: 0.649170, loss: 0.415876
LR 0.0001
val: bce: 0.294598, dice: 0.257134, loss: 0.275866
saving best model to /content/drive/My Drive/intro_ml/unet_train_8_epochs.pt
78m 11s
Epoch 4/7
----------
train: bce: 0.250561, dice: 0.303994, loss: 0.277277
LR 0.0001
val: bce: 0.298350, dice: 

Functions to plot images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_img_array(img_array, ncol=3):
    nrow = len(img_array) // ncol

    f, plots = plt.subplots(nrow, ncol, sharex='all', sharey='all', figsize=(ncol * 4, nrow * 4))

    for i in range(len(img_array)):
        plots[i // ncol, i % ncol]
        plots[i // ncol, i % ncol].imshow(img_array[i])

from functools import reduce
def plot_side_by_side(img_arrays):
    flatten_list = reduce(lambda x,y: x+y, zip(*img_arrays))
    plot_img_array(np.array(flatten_list), ncol=len(img_arrays))


Left: Input image, Middle: Correct mask (Ground-truth), Rigth: Predicted mask

In [ ]:
plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()]) #


NameError: ignored

## Train 3

In [8]:
class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=True)
    
    avg_weights = torch.mean(self.base_model.conv1.weight, 1, True)
    self.base_model.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)
    self.base_model.conv1.weight = nn.Parameter(avg_weights)
    
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = conv_relu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = conv_relu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = conv_relu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = conv_relu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = conv_relu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = conv_relu(256 + 512, 512, 3, 1)
    self.conv_up2 = conv_relu(128 + 512, 256, 3, 1)
    self.conv_up1 = conv_relu(64 + 256, 256, 3, 1)
    self.conv_up0 = conv_relu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = conv_relu(1, 64, 3, 1)
    self.conv_original_size1 = conv_relu(64, 64, 3, 1)
    self.conv_original_size2 = conv_relu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out

In [9]:
model = ResNetUNet(1).to("cuda:0")
criterion = nn.BCEWithLogitsLoss()

for l in model.base_layers:
  for param in l.parameters():
    param.requires_grad = False


import torch.optim as optim
from torch.optim import lr_scheduler
 
optim = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [10]:
# Test inference
timer = time.time()
for epoch in range(N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 2418.16
